In [125]:
import os, sys, time
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from astropy.io import fits
from collections import Counter

In [2]:
table = pd.read_csv('/home/shichenhui/code/data/spectra_table_both_sdss.csv')
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229639 entries, 0 to 229638
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   obsid       229639 non-null  int64  
 1   ra_lamost   229639 non-null  float64
 2   dec_lamost  229639 non-null  float64
 3   specObjID   229639 non-null  float64
 4   plate       229639 non-null  int64  
 5   mjd         229639 non-null  int64  
 6   fiberID     229639 non-null  int64  
 7   ra          229639 non-null  float64
 8   dec         229639 non-null  float64
 9   class       229639 non-null  object 
 10  subClass    155432 non-null  object 
 11  sn1_g       229639 non-null  float64
 12  sn2_g       229639 non-null  float64
 13  sn1_i       229639 non-null  float64
 14  sn2_i       229639 non-null  float64
dtypes: float64(9), int64(4), object(2)
memory usage: 26.3+ MB


In [5]:
table['subClass']

0          STARFORMING
1          STARFORMING
2          STARFORMING
3                  NaN
4                  NaN
              ...     
229634       BROADLINE
229635       BROADLINE
229636       BROADLINE
229637    F0IV (81937)
229638    F0IV (81937)
Name: subClass, Length: 229639, dtype: object

In [6]:
table_lamost = pd.read_csv('/home/shichenhui/code/data/dr8_gb_greater_45.csv')
table_lamost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2586087 entries, 0 to 2586086
Data columns (total 51 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Unnamed: 0       int64  
 1   obsid            int64  
 2   uid              object 
 3   gp_id            int64  
 4   designation      object 
 5   obsdate          object 
 6   lmjd             int64  
 7   mjd              int64  
 8   planid           object 
 9   spid             int64  
 10  fiberid          int64  
 11  ra_obs           float64
 12  dec_obs          float64
 13  snru             float64
 14  snrg             float64
 15  snrr             float64
 16  snri             float64
 17  snrz             float64
 18  objtype          object 
 19  class            object 
 20  subclass         object 
 21  z                float64
 22  z_err            float64
 23  magtype          object 
 24  mag1             float64
 25  mag2             float64
 26  mag3             float64
 27  mag4        

In [40]:

table['class_lamost'] = ''
table['subclass_lamost'] = ''
table['filename_sdss'] = ''
table['filename_lamost'] = ''


In [41]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229639 entries, 0 to 229638
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   obsid            229639 non-null  int64  
 1   ra_lamost        229639 non-null  float64
 2   dec_lamost       229639 non-null  float64
 3   specObjID        229639 non-null  float64
 4   plate            229639 non-null  int64  
 5   mjd              229639 non-null  int64  
 6   fiberID          229639 non-null  int64  
 7   ra               229639 non-null  float64
 8   dec              229639 non-null  float64
 9   class            229639 non-null  object 
 10  subClass         155432 non-null  object 
 11  sn1_g            229639 non-null  float64
 12  sn2_g            229639 non-null  float64
 13  sn1_i            229639 non-null  float64
 14  sn2_i            229639 non-null  float64
 15  class_lamost     229639 non-null  object 
 16  subclass_lamost  229639 non-null  obje

In [21]:
def parse_num(s, length):
    s = str(s)
    l = len(s)
    return '0' * (length - l) + s

In [18]:
folder_sdss = '/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/'
folder_lamost = '/home/shichenhui/code/data/spectra_gb_greater_45/'

In [86]:

for index, row_sdss in tqdm(table.iterrows()):
    #print(index)
    if row_sdss['class'] == 'STAR':
        
        row_lamost = table_lamost.loc[table_lamost['obsid']==row_sdss['obsid']].iloc[0]
        #print(row_lamost['class'],row_lamost['subclass'])
        table.loc[index, 'class_lamost'] = row_lamost['class']

        table.loc[index, 'subclass_lamost'] = row_lamost['subclass']
        f_sdss = 'spec-%s-%s-%s.fits' % (
                    parse_num(row_sdss['plate'], 4), parse_num(row_sdss['mjd'], 5), parse_num(row_sdss['fiberID'], 4))
        f_lamost = 'spec-' + str(row_lamost['lmjd']) + '-' + str(row_lamost['planid']) + '_sp' + parse_num(str(row_lamost['spid']), 2) + '-' + \
                   parse_num(str(row_lamost['fiberid']), 3) + '.fits.gz'

        if os.path.exists(folder_sdss+f_sdss):
            #print(folder_sdss+f_sdss)
            table.loc[index, 'filename_sdss'] = f_sdss
        else:
            table.loc[index, 'filename_sdss'] = np.NAN
        if os.path.exists(folder_lamost+f_lamost):
            table.loc[index, 'filename_lamost'] = f_lamost
        else:
            table.loc[index, 'filename_lamost'] = np.NAN
        #print(row_sdss)

0it [00:00, ?it/s]

In [81]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229639 entries, 0 to 229638
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   obsid            229639 non-null  int64  
 1   ra_lamost        229639 non-null  float64
 2   dec_lamost       229639 non-null  float64
 3   specObjID        229639 non-null  float64
 4   plate            229639 non-null  int64  
 5   mjd              229639 non-null  int64  
 6   fiberID          229639 non-null  int64  
 7   ra               229639 non-null  float64
 8   dec              229639 non-null  float64
 9   class            229639 non-null  object 
 10  subClass         155432 non-null  object 
 11  sn1_g            229639 non-null  float64
 12  sn2_g            229639 non-null  float64
 13  sn1_i            229639 non-null  float64
 14  sn2_i            229639 non-null  float64
 15  class_lamost     229639 non-null  object 
 16  subclass_lamost  229639 non-null  obje

In [87]:
#table1 = table.copy()
#table.loc[1, 'class_lamost']='wer'
table

,obsid,ra_lamost,dec_lamost,specObjID,plate,mjd,fiberID,ra,dec,class,subClass,sn1_g,sn2_g,sn1_i,sn2_i,class_lamost,subclass_lamost,filename_sdss,filename_lamost
0,319612101,155.7512,-0.059962,3.051280e+17,271,51883,33,155.75117,-0.059968,GALAXY,STARFORMING,20.1005,21.8072,18.3134,17.2228,GALAXY,Non,spec-0271-51883-0033.fits,spec-57070-HD101607S013552M01_sp12-101.fits.gz
1,229902160,155.1911,0.410111,3.052701e+17,271,51883,550,155.19108,0.410123,GALAXY,STARFORMING,20.1005,21.8072,18.3134,17.2228,GALAXY,Non,spec-0271-51883-0550.fits,spec-56742-HD102942N012928B01_sp02-160.fits.gz
2,315412029,155.1911,0.410111,3.052701e+17,271,51883,550,155.19108,0.410123,GALAXY,STARFORMING,20.1005,21.8072,18.3134,17.2228,STAR,F6,spec-0271-51883-0550.fits,spec-57062-HD101607S013552B01_sp12-029.fits.gz
3,319612183,155.6168,0.284210,3.052929e+17,271,51883,633,155.61682,0.284189,GALAXY,NaN,20.1005,21.8072,18.3134,17.2228,Unknown,Non,spec-0271-51883-0633.fits,spec-57070-HD101607S013552M01_sp12-183.fits.gz
4,134812198,155.5197,0.069353,3.063171e+17,272,51941,263,155.51972,0.069365,GALAXY,NaN,20.0803,21.2267,20.0486,20.1782,Unknown,Non,spec-0272-51941-0263.fits,spec-56365-HD101607S013552F01_sp12-198.fits.gz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229634,523514070,200.1005,31.071750,1.283658e+19,11401,58491,713,200.10050,31.071752,QSO,BROADLINE,10.6011,13.2886,24.6392,26.4406,,,,
229635,237407075,200.1500,30.903710,1.283658e+19,11401,58491,720,200.15003,30.903717,QSO,BROADLINE,10.6011,13.2886,24.6392,26.4406,,,,
229636,523514062,200.1500,30.903710,1.283658e+19,11401,58491,720,200.15003,30.903717,QSO,BROADLINE,10.6011,13.2886,24.6392,26.4406,,,,
229637,573006207,200.4267,31.582610,1.283659e+19,11401,58491,752,200.42659,31.582603,STAR,F0IV (81937),10.6011,13.2886,24.6392,26.4406,STAR,F0,NaN,spec-57891-HD131344N323149M02_sp06-207.fits.gz


In [96]:
class_num = {'A':0,'F':0,'G':0,'K':0}
for index, row_sdss in tqdm(table.iterrows()):
    if row_sdss['class']==row_sdss['class_lamost']=='STAR':
        if row_sdss['subClass'][0]==row_sdss['subclass_lamost'][0]:
            if row_sdss['subClass'][0] in ['A','F','G','K']:
                if isinstance(row_sdss['filename_sdss'],str) and isinstance(row_sdss['filename_lamost'],str):
                    if os.path.exists(folder_sdss+row_sdss['filename_sdss']) and os.path.exists(folder_lamost+row_sdss['filename_lamost']):
                        class_num[row_sdss['subClass'][0]] += 1

0it [00:00, ?it/s]

In [97]:
class_num

{'A': 5824, 'F': 5380, 'G': 4151, 'K': 6240}

In [98]:
def read_fits(fits_path):
    fits_file = fits.open(fits_path)
    hdu = fits_file[0]
    data = hdu.data[0]

    coeff0 = hdu.header['COEFF0']

    start = round(np.log10(4000), 4)
    connect1 = round(np.log10(5700), 4)
    connect2 = round(np.log10(5900), 4)
    end = round(np.log10(8510), 4)

    start_index = int((start - coeff0) / 0.0001)
    connect1_index = int((connect1 - coeff0) / 0.0001)
    connect2_index = int((connect2 - coeff0) / 0.0001)
    end_index = int((end - coeff0) / 0.0001)

    flux = np.concatenate((data[start_index: connect1_index], data[connect2_index: end_index]), axis=0)

    fits_file.close()
    # print(flux.shape)

    # if flux.shape[0] != 3121:
    #     raise ValueError

    return flux[:3121]

In [131]:
def read_fits_sdss(fits_path):
    fits_file = fits.open(fits_path)
    hdu = fits_file[0]
    data = fits_file[1].data.field('FLUX')

    coeff0 = hdu.header['COEFF0']

    start = round(np.log10(4000), 4)

    end = round(np.log10(8770), 4)

    start_index = int((start - coeff0) / 0.0001)
    end_index = int((end - coeff0) / 0.0001)

    flux = data[start_index: end_index]

    fits_file.close()
    # print(flux.shape)

    if flux.shape[0] < 3405:
        raise ValueError

    return flux[:3405]

In [132]:
class_num = {'A':0,'F':1,'G':2,'K':3}
data_lamost = []
data_sdss = []
for index, row_sdss in tqdm(table.iterrows()):
    if row_sdss['class']==row_sdss['class_lamost']=='STAR':
        if row_sdss['subClass'][0]==row_sdss['subclass_lamost'][0]:
            if row_sdss['subClass'][0] in ['A','F','G','K']:
                if isinstance(row_sdss['filename_sdss'],str) and isinstance(row_sdss['filename_lamost'],str):
                    if os.path.exists(folder_sdss+row_sdss['filename_sdss']) and os.path.exists(folder_lamost+row_sdss['filename_lamost']):
                        try:
                            sp_i = read_fits(folder_lamost+row_sdss['filename_lamost'])
                            sp_i = np.append(sp_i, class_num[row_sdss['subClass'][0]])
                            data_lamost.append(sp_i)
                            sp_j = read_fits_sdss(folder_sdss+row_sdss['filename_sdss'])
                            sp_j = np.append(sp_j, class_num[row_sdss['subClass'][0]])
                            data_sdss.append(sp_j)
                        except:
                            print(folder_sdss+row_sdss['filename_sdss'])
                        

0it [00:00, ?it/s]

/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-0323-51615-0176.fits


/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-0340-51990-0581.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-0340-51990-0581.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-0503-51999-0180.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-0503-51999-0180.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-0504-52316-0278.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-0536-52024-0176.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-0536-52024-0176.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-0538-52029-0523.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-0558-52317-0523.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-0612-52079-0250.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-0878-52353-0551.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-0

/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-2116-53854-0386.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-2137-54206-0640.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-2138-53757-0545.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-2176-54243-0250.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-2297-53738-0505.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-2353-53794-0176.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-2393-54156-0176.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-2393-54156-0176.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-2393-54156-0176.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-2406-54084-0176.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-2406-54084-0176.fits
/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-2

In [133]:
data_lamost_array = np.array(data_lamost)
data_sdss_array = np.array(data_sdss)

In [134]:
data_lamost_array.shape

(21595, 3122)

In [135]:
Counter(data_lamost_array[:,-1])

Counter({0.0: 5824, 2.0: 4151, 1.0: 5380, 3.0: 6240})

In [136]:
data_sdss_array.shape

(21525, 3406)

In [137]:
Counter(data_sdss_array[:,-1])

Counter({0.0: 5797, 2.0: 4144, 1.0: 5355, 3.0: 6229})

In [138]:
f_save = open(r'/home/shichenhui/code/data/data_process/constract_dataset/both_lamost.csv', 'w')
np.savetxt(f_save, data_lamost_array, fmt='%.4f', delimiter=',')
f_save.close()

In [139]:
f_save = open(r'/home/shichenhui/code/data/data_process/constract_dataset/both_sdss.csv', 'w')
np.savetxt(f_save, data_sdss_array, fmt='%.4f', delimiter=',')
f_save.close()

In [118]:
# 测试读取sdss光谱
fits_path = '/home/shichenhui/code/data/spectra_bl_greater_45_both_sdss/spec-0271-51883-0633.fits'
fits_f=fits.open(fits_path)


fits_f[1].data,np.array(fits_f[1].data)

(FITS_rec([( 0.65092945, 3.5837, 0.13239732, 0, 0, 1.1027   , 11.491563 ,  3.2980127),
           ( 5.4315395 , 3.5838, 0.13300723, 0, 0, 1.1027446,  9.73197  ,  3.3170137),
           ( 1.7685558 , 3.5839, 0.14442177, 0, 0, 1.1023489,  8.947948 ,  3.1808774),
           ...,
           (11.349359  , 3.9644, 0.4301919 , 0, 0, 0.762726 ,  5.6773763, 11.885662 ),
           (10.122386  , 3.9645, 0.40675002, 0, 0, 0.7634966,  7.4592633, 11.880726 ),
           ( 9.011201  , 3.9646, 0.39071622, 0, 0, 0.7642198,  8.992491 , 11.907731 )],
          dtype=(numpy.record, [('flux', '>f4'), ('loglam', '>f4'), ('ivar', '>f4'), ('and_mask', '>i4'), ('or_mask', '>i4'), ('wdisp', '>f4'), ('sky', '>f4'), ('model', '>f4')])),
 3.5837)